## Bulletines Cleaning

I used this notebook to clean raw data previously scraped containing information from the monthly bulletines published by the Italian Ministry of Justice.

In [1]:
import pandas as pd
import numpy as np
import re

### Basic cleaning

- Converting numbers into integers (note that the `.` in Italy is being used as a decimal separator)
- Removing `Totals` (we'll calculate them when needed)
- Fixing empty values to numeric (from NaN to 0)
- Converting to integer

In [2]:

# Read CSV with thousands separator specified
df = pd.read_csv("../outputs/raw/bulletines_raw.csv", thousands=".")

# Remove rows where "Regione di detenzione" is "Totale"
df = df[df["Regione di detenzione"] != "Totale"]

# Replace empty strings and similar with NaN
df.replace(["", " ", "NaN", "nan"], np.nan, inplace=True)

# Fill NaN values with 0
df.fillna(0, inplace=True)

# Convert columns to integer type
df['Detenuti presenti - stranieri'] = df['Detenuti presenti - stranieri'].astype(int)

# Remove dots from numeric strings in the specified column
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].str.replace(".", "")

# Show a random sample of 5 rows
df.sample(5)


Regione di detenzione Sigla Provincia              Istituto  \
12705               TOSCANA              GR              GROSSETO   
59                    LAZIO              RM  CIVITAVECCHIA "N.C."   
3010                TOSCANA              GR            GROSSETO -   
5053                 PUGLIA              BA              ALTAMURA   
13775             LOMBARDIA              VA         BUSTO ARSIZIO   

      Tipo istituto  Capienza Regolamentare Detenuti presenti - totale  \
12705            CC                      15                         28   
59               CC                     357                        539   
3010             CC                      15                         31   
5053             CR                      53                         81   
13775            CC                     240                        413   

      Detenuti presenti - donne  Detenuti presenti - stranieri  \
12705                         0                             10   
59                           43                            264   
3010                          0                             17   
5053                          0                              4   
13775                         0                            210   

      Ultimo aggiornamento          ID  
12705           2019-02-28   SST173677  
59              2024-08-31  SST1418233  
3010            2023-05-31   SST431165  
5053            2022-06-30   SST386394  
13775           2024-12-31  SST1437082

In [3]:
df.dtypes

Regione di detenzione            object
Sigla Provincia                  object
Istituto                         object
Tipo istituto                    object
Capienza Regolamentare            int64
Detenuti presenti - totale       object
Detenuti presenti - donne        object
Detenuti presenti - stranieri     int64
Ultimo aggiornamento             object
ID                               object
dtype: object

In [4]:
# Step 1: Replace dots in the 'Detenuti presenti - totale' column, then convert to numeric
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].str.replace(".", "").astype(float)
# Step 2: Fill any remaining NaN values with 0
df['Detenuti presenti - totale'].fillna(0, inplace=True)
# Step 3: Convert the cleaned column to integers
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].astype(int)
df['Detenuti presenti - donne'] = df['Detenuti presenti - donne'].astype(int)


/tmp/ipykernel_3078/1983749719.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Detenuti presenti - totale'].fillna(0, inplace=True)


In [5]:
df.sort_values('Detenuti presenti - stranieri', ascending=False).head(15)

Regione di detenzione Sigla Provincia  \
13577             LOMBARDIA              MI   
13197             LOMBARDIA              MI   
13387             LOMBARDIA              MI   
2010               PIEMONTE              TO   
1221              LOMBARDIA              MI   
1820               PIEMONTE              TO   
1630               PIEMONTE              TO   
651               LOMBARDIA              MI   
1411              LOMBARDIA              MI   
300                PIEMONTE              TO   
1440               PIEMONTE              TO   
13957             LOMBARDIA              MI   
841               LOMBARDIA              MI   
490                PIEMONTE              TO   
11510              PIEMONTE              TO   

                                           Istituto Tipo istituto  \
13577            MILANO "F. DI CATALDO" SAN VITTORE            CC   
13197            MILANO "F. DI CATALDO" SAN VITTORE            CC   
13387            MILANO "F. DI CATALDO" SAN VITTORE            CC   
2010   TORINO "G. LORUSSO - L. CUTUGNO" LE VALLETTE            CC   
1221             MILANO "F. DI CATALDO" SAN VITTORE            CC   
1820   TORINO "G. LORUSSO - L. CUTUGNO" LE VALLETTE            CC   
1630   TORINO "G. LORUSSO - L. CUTUGNO" LE VALLETTE            CC   
651              MILANO "F. DI CATALDO" SAN VITTORE            CC   
1411             MILANO "F. DI CATALDO" SAN VITTORE            CC   
300    TORINO "G. LORUSSO - L. CUTUGNO" LE VALLETTE            CC   
1440   TORINO "G. LORUSSO - L. CUTUGNO" LE VALLETTE            CC   
13957            MILANO "F. DI CATALDO" SAN VITTORE            CC   
841              MILANO "F. DI CATALDO" SAN VITTORE            CC   
490    TORINO "G. LORUSSO - L. CUTUGNO" LE VALLETTE            CC   
11510    TORINO "G. LORUSSO L. CUTUGNO" LE VALLETTE            CC   

       Capienza Regolamentare  Detenuti presenti - totale  \
13577                     748                        1162   
13197                     748                        1149   
13387                     748                        1149   
2010                     1118                        1496   
1221                      754                        1146   
1820                     1118                        1480   
1630                     1118                        1491   
651                       754                        1150   
1411                      754                        1158   
300                      1117                        1494   
1440                     1118                        1444   
13957                     748                        1084   
841                       754                        1126   
490                      1117                        1478   
11510                    1061                        1516   

       Detenuti presenti - donne  Detenuti presenti - stranieri  \
13577                         85                            743   
13197                         83                            732   
13387                         83                            732   
2010                         126                            719   
1221                          85                            716   
1820                         125                            714   
1630                         126                            714   
651                           85                            709   
1411                          81                            708   
300                          135                            706   
1440                         128                            698   
13957                         86                            696   
841                           82                            695   
490                          131                            694   
11510                        144                            694   

      Ultimo aggiornamento          ID  
13577           2024-11-30  SST1433924  
13197           

In [6]:
df.dtypes

Regione di detenzione            object
Sigla Provincia                  object
Istituto                         object
Tipo istituto                    object
Capienza Regolamentare            int64
Detenuti presenti - totale        int64
Detenuti presenti - donne         int64
Detenuti presenti - stranieri     int64
Ultimo aggiornamento             object
ID                               object
dtype: object

In [7]:
# Convert the 'Ultimo aggiornamento' column to datetime format
df['Ultimo aggiornamento'] = pd.to_datetime(df['Ultimo aggiornamento'])


In [8]:
df.dtypes

Regione di detenzione                    object
Sigla Provincia                          object
Istituto                                 object
Tipo istituto                            object
Capienza Regolamentare                    int64
Detenuti presenti - totale                int64
Detenuti presenti - donne                 int64
Detenuti presenti - stranieri             int64
Ultimo aggiornamento             datetime64[ns]
ID                                       object
dtype: object

### Fixing names

A problem we are encountering are the many different names each detention center has been registered with over the years. Here below we use [thefuzz](https://github.com/seatgeek/thefuzz) to do an initial fuzzy matching, and then fix the remaining ones manually. As a reference, we'll use the information we have scraped in a different notebook about all the detention centers currently operating in Italy.

In [9]:
df['Istituto'] = df['Istituto'].str.strip()
df['Istituto'] = df['Istituto'].str.replace(r"\s*-", "", regex=True)  # Removes any whitespace followed by a dash


In [10]:
sorted(df['Istituto'].unique())

['AGRIGENTO "P. DI LORENZO"',
 'AGRIGENTO "PASQUALE DI LORENZO"',
 'ALBA "G. MONTALTO"',
 'ALBA "GIUSEPPE MONTALTO"',
 'ALESSANDRIA "G. CANTIELLO  S. GAETA"',
 'ALESSANDRIA "G. CANTIELLO S. GAETA"',
 'ALESSANDRIA "SAN MICHELE"',
 'ALGHERO "G. TOMASIELLO"',
 'ALGHERO "GIUSEPPE TOMASIELLO"',
 'ALTAMURA',
 'ANCONA',
 'ANCONA "BARCAGLIONE"',
 'ARBUS "IS ARENAS"',
 'AREZZO',
 'ARIANO IRPINO "P. CAMPANELLO"',
 'ARIANO IRPINO "PASQUALE CAMPANELLO"',
 'ARIENZO',
 'ARIENZO "G. DE ANGELIS"',
 'ARIENZO "GENNARO DE ANGELIS"',
 'ASCOLI PICENO',
 'ASTI',
 'AUGUSTA',
 'AVELLINO "A. GRAZIANO" BELLIZZI',
 'AVELLINO "ANTIMO GRAZIANO" BELLIZZI',
 'AVERSA "F. SAPORITO"',
 'AVEZZANO',
 'BARCELLONA POZZO DI GOTTO',
 'BARI "F. RUCCI"',
 'BARI "FRANCESCO RUCCI"',
 'BELLUNO',
 'BENEVENTO',
 'BENEVENTO "M. GAGLIONE"',
 'BERGAMO',
 'BERGAMO "Don Fausto RESMINI"',
 'BIELLA',
 'BOLLATE "II C.R."',
 'BOLOGNA "R. D\'AMATO"',
 'BOLOGNA "ROCCO D\'AMATO"',
 'BOLZANO',
 'BRESCIA "N. FISCHIONE" CANTON MONBELLO',
 'BRESCI

In [11]:
from thefuzz import process

In [12]:
# Import standard names
df_standard_names = pd.read_csv('../outputs/clean/institutes_info.csv')
standard_names = df_standard_names['nome_istituto'].tolist()
standard_names

['Reggio Calabria Arghillà',
 'Brescia Verziano',
 'Busto Arsizio',
 'Como',
 'Cremona',
 'Lecco',
 'Lodi',
 'Mantova',
 'Monza',
 'Pavia',
 'Sondrio',
 'Varese',
 'Voghera',
 'Catanzaro',
 'Crotone',
 'Palmi',
 'Paola',
 'Rossano',
 'Vibo Valentia',
 'Ariano Irpino',
 'Aversa',
 'Carinola',
 'Eboli',
 'Bolzano',
 'Gorizia',
 'Padova',
 'Santa Maria Capua Vetere',
 'Pordenone',
 'Rovigo',
 'Ascoli Piceno',
 'Castelfranco Emilia',
 'Fermo',
 'Treviso',
 'Fossombrone',
 'Trieste',
 'Udine',
 'Pesaro',
 'Venezia Santa Maria Maggiore',
 'Venezia Giudecca',
 'Verona',
 'Avezzano',
 'Campobasso',
 'Alba',
 'Cassino',
 'Chieti',
 'Asti',
 'Civitavecchia Giuseppe Passerini',
 'Chiavari',
 'Frosinone',
 'Cuneo',
 'Trento',
 'Brissogne - Aosta',
 'Isernia',
 'Genova Pontedecimo',
 'Larino',
 'Imperia',
 'Ivrea',
 'Paliano',
 'La Spezia',
 'Rieti',
 'Roma Regina Coeli',
 'Saluzzo',
 'Roma Rebibbia',
 'Roma Rebibbia IIIª casa',
 'Roma Rebibbia femminile',
 'Verbania',
 'Padova Nuovo Complesso',
 '

In [13]:
df_copy = df
df_copy.head(4)

Regione di detenzione Sigla Provincia  Istituto Tipo istituto  \
0               ABRUZZO              AQ  AVEZZANO            CC   
1               ABRUZZO              AQ  L'AQUILA            CC   
2               ABRUZZO              AQ   SULMONA            CR   
3               ABRUZZO              CH    CHIETI            CC   

   Capienza Regolamentare  Detenuti presenti - totale  \
0                      53                          71   
1                     228                         166   
2                     323                         443   
3                      79                         140   

   Detenuti presenti - donne  Detenuti presenti - stranieri  \
0                          0                             24   
1                         12                             15   
2                          0                             10   
3                         36                             35   

  Ultimo aggiornamento          ID  
0           2024-08-31  SST1418233  
1           2024-08-31  SST1418233  
2           2024-08-31  SST1418233  
3           2024-08-31  SST1418233

In [14]:
def standardize_name(name):
    match, score = process.extractOne(name, standard_names)
    return match if score >= 90 else name  # Adjust score threshold as needed
# Apply fuzzy matching to standardize names
df_copy['Istituto'] = df_copy['Istituto'].apply(standardize_name)

In [15]:
sorted(df_copy['Istituto'].unique())

['Agrigento',
 'Alba',
 'Alessandria Cantiello, Gaeta',
 'Alessandria San Michele',
 'Alghero',
 'Altamura',
 'Ancona Barcaglione',
 'Arbus',
 'Arezzo',
 'Ariano Irpino',
 'Arienzo',
 'Ascoli Piceno',
 'Asti',
 'Augusta',
 'Avellino',
 'Aversa',
 'Avezzano',
 'BRESCIA "N. FISCHIONE" CANTON MONBELLO',
 'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO',
 'Barcellona Pozzo di Gotto',
 'Bari',
 'Belluno',
 'Benevento',
 'Bergamo',
 'Biella',
 'Bollate',
 'Bologna',
 'Bolzano',
 'Brescia Verziano',
 'Brindisi',
 'Brissogne - Aosta',
 'Busto Arsizio',
 'CAGLIARI "E.SCALAS"',
 'CAGLIARI "ETTORE SCALAS"',
 'CAMERINO',
 'CIVITAVECCHIA "N.C."',
 'Caltagirone',
 'Caltanissetta',
 'Campobasso',
 'Carinola',
 'Cassino',
 'Castelfranco Emilia',
 'Castelvetrano',
 'Castrovillari',
 'Catania Bicocca',
 'Catania Piazza Lanza',
 'Catanzaro',
 'Chiavari',
 'Chieti',
 'Civitavecchia Giuseppe Passerini',
 'Como',
 'Cosenza',
 'Cremona',
 'Crotone',
 'Cuneo',
 'Eboli',
 'Enna',
 "FORLI'",
 'Favignana',
 'Fermo',


Because the names that did not match are uppercase, we can identify them easily and proceed with a mapping.

In [16]:
sorted(df_copy[df_copy['Istituto'].str.isupper()]['Istituto'].unique())

['BRESCIA "N. FISCHIONE" CANTON MONBELLO',
 'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO',
 'CAGLIARI "E.SCALAS"',
 'CAGLIARI "ETTORE SCALAS"',
 'CAMERINO',
 'CIVITAVECCHIA "N.C."',
 "FORLI'",
 'MILANO "F. DI CATALDO" SAN VITTORE',
 'MILANO "FRANCESCO DI CATALDO" SAN VITTORE',
 'NAPOLI "G. SALVIA" POGGIOREALE',
 'NAPOLI "GIUSEPPE SALVIA" POGGIOREALE',
 'NAPOLI "P. MANDATO" SECONDIGLIANO',
 'NAPOLI "PASQUALE MANDATO" SECONDIGLIANO',
 'PALERMO "A. LORUSSO" PAGLIARELLI',
 'PALERMO "ANTONIO LORUSSO" PAGLIARELLI',
 'PALERMO "C. DI BONA" UCCIARDONE',
 'PALERMO "CALOGERO DI BONA" UCCIARDONE',
 'ROMA "G. STEFANINI" REBIBBIA FEMMINILE',
 'ROMA "GERMANA STEFANINI" REBIBBIA FEMMINILE',
 'ROMA "R. CINOTTI" REBIBBIA N.C.1',
 'ROMA "RAFFAELE CINOTTI" REBIBBIA N.C.1',
 'ROMA "REBIBBIA TERZA CASA"',
 'SAN REMO "N.C."']

In [17]:
institute_mapping = {
    'BRESCIA "N. FISCHIONE" CANTON MONBELLO': 'Brescia Canton Monbello',
    'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO': 'Brescia Canton Monbello',
    'CAGLIARI "E.SCALAS"': 'Cagliari Uta',
    'CAGLIARI "ETTORE SCALAS"': 'Cagliari Uta',
    'CAMERINO': 'Camerino',
    'CIVITAVECCHIA "N.C."': 'Civitavecchia Nuovo Complesso',
    "FORLI'": 'Forlì',
    'MILANO "F. DI CATALDO" SAN VITTORE': 'Milano San Vittore',
    'MILANO "FRANCESCO DI CATALDO" SAN VITTORE': 'Milano San Vittore',
    'NAPOLI "G. SALVIA" POGGIOREALE': 'Napoli Poggioreale',
    'NAPOLI "GIUSEPPE SALVIA" POGGIOREALE': 'Napoli Poggioreale',
    'NAPOLI "P. MANDATO" SECONDIGLIANO': 'Napoli Secondigliano',
    'NAPOLI "PASQUALE MANDATO" SECONDIGLIANO': 'Napoli Secondigliano',
    'PALERMO "A. LORUSSO" PAGLIARELLI': 'Palermo Pagliarelli',
    'PALERMO "ANTONIO LORUSSO" PAGLIARELLI': 'Palermo Pagliarelli',
    'PALERMO "C. DI BONA" UCCIARDONE': 'Palermo Ucciardone',
    'PALERMO "CALOGERO DI BONA" UCCIARDONE': 'Palermo Ucciardone',
    'ROMA "G. STEFANINI" REBIBBIA FEMMINILE': 'Roma Rebibbia Femminile',
    'ROMA "GERMANA STEFANINI" REBIBBIA FEMMINILE': 'Roma Rebibbia Femminile',
    'ROMA "R. CINOTTI" REBIBBIA N.C.1': 'Roma Rebibbia',
    'ROMA "RAFFAELE CINOTTI" REBIBBIA N.C.1': 'Roma Rebibbia',
    'ROMA "REBIBBIA TERZA CASA"': 'Roma Rebibbia III Casa',
    'SAN REMO "N.C."': 'Sanremo',
}

df_copy['Istituto'] = df_copy['Istituto'].replace(institute_mapping)

# Check the updated unique values
df_copy['Istituto'].unique()

array(['Avezzano', "L'Aquila", 'Sulmona', 'Chieti', 'Lanciano', 'Vasto',
       'Pescara', 'Teramo', 'Matera', 'Melfi', 'Potenza', 'Castrovillari',
       'Cosenza', 'Paola', 'Rossano', 'Catanzaro', 'Crotone',
       'Laureana di Borrello', 'Locri', 'Palmi',
       'Reggio Calabria Arghillà', 'Reggio Calabria Giuseppe Panzera',
       'Vibo Valentia', 'Ariano Irpino', 'Avellino', 'Lauro',
       "Sant'Angelo dei Lombardi", 'Benevento', 'Arienzo', 'Aversa',
       'Carinola', 'Santa Maria Capua Vetere', 'Napoli Poggioreale',
       'Napoli Secondigliano', 'Pozzuoli', 'Eboli', 'Salerno',
       'Vallo della Lucania', 'Bologna', 'Ferrara', 'Forlì',
       'Castelfranco Emilia', 'Modena', 'Piacenza', 'Parma', 'Ravenna',
       'Reggio Emilia', 'Rimini', 'Gorizia', 'Pordenone', 'Trieste',
       'Tolmezzo', 'Udine', 'Cassino', 'Frosinone', 'Paliano', 'Latina',
       'Rieti', 'Civitavecchia Giuseppe Passerini',
       'Civitavecchia Nuovo Complesso', 'Roma Rebibbia Femminile',
       'Roma 

In [18]:
df_copy.sample(5)

Regione di detenzione Sigla Provincia                 Istituto  \
4636               LOMBARDIA              CO                     Como   
11193    TRENTINO ALTO ADIGE              BZ                  Bolzano   
12016         EMILIA ROMAGNA              RA                  Ravenna   
8271                PIEMONTE              AL  Alessandria San Michele   
8031   FRIULI VENEZIA GIULIA              UD                 Tolmezzo   

      Tipo istituto  Capienza Regolamentare  Detenuti presenti - totale  \
4636             CC                     240                         362   
11193            CC                      87                         118   
12016            CC                      49                          87   
8271             CR                     267                         310   
8031             CC                     149                         195   

       Detenuti presenti - donne  Detenuti presenti - stranieri  \
4636                          40                            204   
11193                          0                             88   
12016                          0                             47   
8271                           0                            148   
8031                           0                             22   

      Ultimo aggiornamento         ID  
4636            2022-08-31  SST393944  
11193           2019-10-31  SST225673  
12016           2019-05-31  SST193650  
8271            2021-01-31  SST319895  
8031            2021-02-28  SST323234

In [19]:
# # remove duplicate data from Dec 2021
# df_copy = df_copy[df_copy['ID'] != 'SST360932']

In [20]:
df_copy = df_copy.sort_values(['Ultimo aggiornamento'], ascending=False)
df_copy.tail()

Regione di detenzione Sigla Provincia            Istituto Tipo istituto  \
12801                 LAZIO              VT             Viterbo            CC   
12802               LIGURIA              GE            Chiavari            CR   
12803               LIGURIA              GE      Genova Marassi            CC   
12804               LIGURIA              GE  Genova Pontedecimo            CC   
12737               ABRUZZO              AQ             Sulmona            CR   

       Capienza Regolamentare  Detenuti presenti - totale  \
12801                     432                         548   
12802                      45                          42   
12803                     546                         727   
12804                      96                         145   
12737                     304                         368   

       Detenuti presenti - donne  Detenuti presenti - stranieri  \
12801                          0                            289   
12802                          0                             14   
12803                          0                            404   
12804                         73                             61   
12737                          0                             10   

      Ultimo aggiornamento         ID  
12801           2019-01-31  SST168760  
12802           2019-01-31  SST168760  
12803           2019-01-31  SST168760  
12804           2019-01-31  SST168760  
12737           2019-01-31  SST168760

In [21]:
df_copy.to_csv('../outputs/clean/bulletines.csv', index=False, encoding="UTF-8-sig")